In [1]:
!pip install --upgrade pip
!pip3 install tensorflow
#!pip3 install tensorflow==2.4.0
!pip install gym
!pip install keras
!pip install keras-rl2

ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.0 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0)
ERROR: No matching distribution found for tensorflow==2.4.0


In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [3]:
class ShowerEnv(Env):
    def __init__(self):
        self.produced_energy = float(input("Energía a producir: "))
        
        #Acctions up, down stay
        self.action_space = Discrete(3)
        #Angle array
        self.observation_space = Box(low=np.array([0]), high=np.array([180]))
        #Set start angle
        self.state = random.randint(0,180)
        #Set reaching energy producted
        self.training_length = 600
        
    def step(self, action):
        #Apply action
        #si la accion es 0, le resta 1 = -1 (temp disminuye 1 grado)
        #si la accion es 1, le resta 1 = 0 (temp ni aumenta ni disminuye)
        #si la accion es 2, le resta 1 = 1 grados (temp aumenta 1 grado)
        self.state += action - 1
        #Reduce el tiempo de ducha en 1 segundo(el total de duracion es 60)
        self.training_length -= 1
        
        #Calcula el reward
        if self.state >=37 and self.state <=39:
            reward = 1
        else:
            reward = -1
        
        #Checkea si la ducha ha terminado
        if self.training_length <=0:
            done = True
        else:
            done = False
            
        #Perturbaciones en el viento
        self.state += random.randint(-10,10)
        #placeholder para la info
        info = {}
        
        #Retorna informacion del paso(step information)
        return self.state, reward, done, info
    
    def render(self):
        #visualización
        pass
    def reset(self):
        #Reseteamos el ángulo
        self.state = random.randint(0,180)
        #Reseteamos el tiempo de entreamiento
        self.training_length = 600
        return self.state

In [4]:
env= ShowerEnv()

C:\Users\dgarr\anaconda3\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:4
Episode:2 Score:-10
Episode:3 Score:-56
Episode:4 Score:-54
Episode:5 Score:-52
Episode:6 Score:-40
Episode:7 Score:-60
Episode:8 Score:-54
Episode:9 Score:10
Episode:10 Score:-4


In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
states = env.observation_space.shape
actions = env.action_space.n

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape = states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [10]:
del model

NameError: name 'model' is not defined

In [11]:
model = build_model(states, actions)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [13]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [14]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)